In [1]:
import pandas as pd
import random as random
import glob, os

#concatener les fichiers en python  et lecture du fichier :
df = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "./DataSet/*.csv"))))

row, col = df.shape
print("Size of the dataframe :" + str(df.size) + " ("+str(row)+"*"+str(col)+")")

df.head()

df

Size of the dataframe :342328 (24452*14)


,id,text,date,truthRating,ratingName,author,headline,named_entities_claim,named_entities_article,keywords,source,sourceURL,link,language
0,http://data.gesis.org/claimskg/claim_review/47...,'Hillary's main extracurricular activity in la...,2008-01-24,1,False,Chain email,"She did not """"help"""" the Black Panthers","ACLU,Alex Rackley,American Civil Liberties Uni...",Black Panthers,Candidate Biography,politifact,http://www.politifact.com,http://www.politifact.com/truth-o-meter/statem...,English
1,http://data.gesis.org/claimskg/claim_review/1d...,Says 'Hillary Clinton's plan would bring in 62...,2016-09-20,1,False,Donald Trump,"Trump says Clinton would bring in 620,000 refu...","Barack Obama,CBS,Center for Global Development...",Hillary Clinton,Immigration,politifact,http://www.politifact.com,http://www.politifact.com/truth-o-meter/statem...,English
2,http://data.gesis.org/claimskg/claim_review/51...,Canada suffered a major donut shortage shortly...,Unknown,1,False,Unknown,Did Canadians Face a Major Donut Shortage Afte...,"CNN,Daily Bonnet,Girl Scout cookies,Tim Horton...",marijuana,"canada, marijuana, world news daily report",snopes,http://www.snopes.com,https://www.snopes.com/fact-check/canada-donut...,English
3,http://data.gesis.org/claimskg/claim_review/c5...,"The mayor of Ath, in Belgium, refused to remov...",Unknown,1,False,Unknown,Did a Belgian Mayor Refuse to Ban Pork in Scho...,"Belgium,Islam,Maryborough, Victoria,Muslim,Que...",NaN,islam,snopes,http://www.snopes.com,https://www.snopes.com/fact-check/belgium-mayo...,English
4,http://data.gesis.org/claimskg/claim_review/ef...,Says Pete Gallego canceled a 'planned meeting ...,2018-07-18,1,False,Roland Gutierrez,Roland Gutierrez errs saying Pete Gallego canc...,"2012 mass shooting,Carlos Uresti,Democrat,Face...",Pete Gallego,"Congress,Guns",politifact,http://www.politifact.com,http://www.politifact.com/texas/statements/201...,English
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4447,http://data.gesis.org/claimskg/claim_review/d7...,A pedestrian was killed by a flying fire hydrant.,Unknown,3,True,Unknown,Fire Hydrant Death,"Associated Press,Encinitas, California,Fire Hy...",fire hydrant,"ASP Article, freakish fatalities",snopes,http://www.snopes.com,https://www.snopes.com/fact-check/fire-plugged/,English
4448,http://data.gesis.org/claimskg/claim_review/b5...,'The United States is the ONLY industrialized ...,2019-04-30,3,True,Mark Pocan,Universal health care diagnosis is on the mark,"Affordable Care Act,Australia,Austria,Belarus,...",NaN,"Health Care,Medicaid,Medicare",politifact,http://www.politifact.com,http://www.politifact.com/wisconsin/statements...,English
4449,http://data.gesis.org/claimskg/claim_review/05...,A 17-year-old Nigerian girl was publicly flogg...,Unknown,3,True,Unknown,Bariya Ibrahim Magazu Petition,"Abuja,Associated Press,Barbara “faith,Dallas M...",premarital sex,"ASP Article, Petitions",snopes,http://www.snopes.com,https://www.snopes.com/fact-check/the-cane-mut...,English
4450,http://data.gesis.org/claimskg/claim_review/7d...,'We haven’t had control of the House for the l...,2010-10-18,3,True,William G. Batchelder,Republican state Rep. Bill Batchelder knocks D...,"Armond Budish,Democrat,PolitiFact,Republican",NaN,Government Efficiency,politifact,http://www.politifact.com,http://www.politifact.com/ohio/statements/2010...,English


In [2]:
#commencement des prétraitement :

#import nécessaire :
import re
import nltk
nltk.download('stopwords') 
nltk.download('punkt')
from nltk.corpus import stopwords
import unicodedata
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer


#La j'ai mis que la colonne text en lower mais je crois qu'on devra tous les mettre par la suite
df['text'] = df['text'].str.lower()


#White spaces removal
df['text'] = df['text'].str.strip()


#Elimination des stopWord :
#input_str = "NLTK is a leading platform for building Python programs to work with human language data."
stop_words = set(stopwords.words("english"))
#La liste des stopwords en Anglais
negation ={'haven''t','cannot',"doesn't","shouldn't","needn't","shant't","weren't","hasn't", "wasn't","didn't", "aren't",'not', "mightn't", "mustn't", 'no',  "wouldn't", "mightn't", "won't",  "needn't", "wasn't", "wouldn't",  "isn't", "doesn't", "weren't", "isn't", "hasn't", "hadn't", "don't", "hadn't","couldn't"}

#La liste des stopwords de négation en Anglais  
from nltk.tokenize import TweetTokenizer
a= df['text'].str.replace("’","'") #Extraire toutes les entrées de la colonne text
pat = r'\b(?:{})\b'.format('|'.join(negation))

a.str.replace(pat,'not')
text_without_stopwords=[] #Une liste dont on va affecter les textes après l'élimination des stop words
tk=TweetTokenizer()
for text in a.iteritems(): #On parcourt toutes les lignes
 tokens = tk.tokenize(str(text[1]))
 result = [i for i in tokens if not i in stop_words-negation]
 splitor=" "
 concatinated = splitor.join(result) #concatiner les tokens
 text_without_stopwords.append(concatinated)

df['text']=text_without_stopwords


#The following code removes this set of symbols [!”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]:
text_without_punctuation=[] # pour y mettre notre résultat
a= df['text'] #Extraire toutes les entrées de la colonne text deja traiter pour faire la suite
tk=TweetTokenizer()
for text in a.iteritems(): #On parcourt toutes les lignes
    result = re.sub('[!”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]', '', str(text[1]) )
    splitor="" #séparateur de mots
    concatinated = splitor.join(result) #concatiner les résultats
    text_without_punctuation.append(concatinated)

df['text']=text_without_punctuation


#The following code removes non ASCII characters :
text_without_ascii=[] # pour y mettre notre résultat
a= df['text'] #Extraire toutes les entrées de la colonne text deja traiter pour faire la suite
tk=TweetTokenizer()
for text in a.iteritems(): #On parcourt toutes les lignes
    text = unicodedata.normalize('NFKD', str(text[1]) ).encode("ascii", "ignore").decode("utf-8", 'ignore')
    splitor="" #séparateur de mots
    concatinated = splitor.join(text)
    text_without_ascii.append(concatinated)

df['text']=text_without_ascii


#Lemmatization :
lemmatizer = WordNetLemmatizer()
text_lemmatizer=[] # pour y mettre notre résultat
a= df['text'] #Extraire toutes les entrées de la colonne text deja traiter pour faire la suite
tk=TweetTokenizer()
for ligne in a.iteritems(): #On parcourt toutes les lignes
    wordList = re.sub("[^\w]", " ",  ligne[1]).split() # on parcourt tout les mot de la ligne
    newList=[]
    for word in wordList:
        if not(word.isdigit()): # si c'est  mot et non un nombre
            newList.append(lemmatizer.lemmatize(word, pos = 'v')) #lemmatisation
    splitor=" " #séparateur de mots
    concatinated = splitor.join(newList)
    text_lemmatizer.append(concatinated)

df['text']=text_lemmatizer
df


#Fin des prétraitement.

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aminb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aminb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aminb\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,id,text,date,truthRating,ratingName,author,headline,named_entities_claim,named_entities_article,keywords,source,sourceURL,link,language
0,http://data.gesis.org/claimskg/claim_review/47...,hillary s main extracurricular activity law sc...,2008-01-24,1,False,Chain email,"She did not """"help"""" the Black Panthers","ACLU,Alex Rackley,American Civil Liberties Uni...",Black Panthers,Candidate Biography,politifact,http://www.politifact.com,http://www.politifact.com/truth-o-meter/statem...,English
1,http://data.gesis.org/claimskg/claim_review/1d...,say hillary clinton s plan would bring refugee...,2016-09-20,1,False,Donald Trump,"Trump says Clinton would bring in 620,000 refu...","Barack Obama,CBS,Center for Global Development...",Hillary Clinton,Immigration,politifact,http://www.politifact.com,http://www.politifact.com/truth-o-meter/statem...,English
2,http://data.gesis.org/claimskg/claim_review/51...,canada suffer major donut shortage shortly cou...,Unknown,1,False,Unknown,Did Canadians Face a Major Donut Shortage Afte...,"CNN,Daily Bonnet,Girl Scout cookies,Tim Horton...",marijuana,"canada, marijuana, world news daily report",snopes,http://www.snopes.com,https://www.snopes.com/fact-check/canada-donut...,English
3,http://data.gesis.org/claimskg/claim_review/c5...,mayor ath belgium refuse remove pork school ca...,Unknown,1,False,Unknown,Did a Belgian Mayor Refuse to Ban Pork in Scho...,"Belgium,Islam,Maryborough, Victoria,Muslim,Que...",NaN,islam,snopes,http://www.snopes.com,https://www.snopes.com/fact-check/belgium-mayo...,English
4,http://data.gesis.org/claimskg/claim_review/ef...,say pete gallego cancel plan meet families san...,2018-07-18,1,False,Roland Gutierrez,Roland Gutierrez errs saying Pete Gallego canc...,"2012 mass shooting,Carlos Uresti,Democrat,Face...",Pete Gallego,"Congress,Guns",politifact,http://www.politifact.com,http://www.politifact.com/texas/statements/201...,English
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4447,http://data.gesis.org/claimskg/claim_review/d7...,pedestrian kill fly fire hydrant,Unknown,3,True,Unknown,Fire Hydrant Death,"Associated Press,Encinitas, California,Fire Hy...",fire hydrant,"ASP Article, freakish fatalities",snopes,http://www.snopes.com,https://www.snopes.com/fact-check/fire-plugged/,English
4448,http://data.gesis.org/claimskg/claim_review/b5...,unite state industrialize country without univ...,2019-04-30,3,True,Mark Pocan,Universal health care diagnosis is on the mark,"Affordable Care Act,Australia,Austria,Belarus,...",NaN,"Health Care,Medicaid,Medicare",politifact,http://www.politifact.com,http://www.politifact.com/wisconsin/statements...,English
4449,http://data.gesis.org/claimskg/claim_review/05...,yearold nigerian girl publicly flog engage pre...,Unknown,3,True,Unknown,Bariya Ibrahim Magazu Petition,"Abuja,Associated Press,Barbara “faith,Dallas M...",premarital sex,"ASP Article, Petitions",snopes,http://www.snopes.com,https://www.snopes.com/fact-check/the-cane-mut...,English
4450,http://data.gesis.org/claimskg/claim_review/7d...,control house last two years malfunction sessi...,2010-10-18,3,True,William G. Batchelder,Republican state Rep. Bill Batchelder knocks D...,"Armond Budish,Democrat,PolitiFact,Republican",NaN,Government Efficiency,politifact,http://www.politifact.com,http://www.politifact.com/ohio/statements/2010...,English


In [58]:
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
from sklearn import svm
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
#On crée des variable LabelEncoder qui vont servir à transférer nos données en valeurs numériques :
l1=LabelEncoder()
l2=LabelEncoder()
l3=LabelEncoder()
l4=LabelEncoder()
l5=LabelEncoder()
l6=LabelEncoder()
l7=LabelEncoder()
l8=LabelEncoder()
l9=LabelEncoder()
l10=LabelEncoder()
l11=LabelEncoder()
l12=LabelEncoder()
l13=LabelEncoder()
#on transforme tous nous données en String car y'avais des entrées qui ont une combinaison du string et float :
df=df.applymap(str) 
#on transfère toutes les valeurs des colonnes qu'on va utiliser en valeurs numériques :
df['id']=l1.fit_transform(df['id'])
df['text']=l2.fit_transform(df['text'])
df['date']=l3.fit_transform(df['date'])
df['author']=l5.fit_transform(df['author'])
df['headline']=l6.fit_transform(df['headline'])
df['named_entities_claim']=l7.fit_transform(df['named_entities_claim'])
df['named_entities_article']=l8.fit_transform(df['named_entities_article'])
df['keywords']=l9.fit_transform(df['keywords'])
df['source']=l10.fit_transform(df['source'])
df['sourceURL']=l11.fit_transform(df['sourceURL'])
df['link']=l12.fit_transform(df['link'])
df['language']=l13.fit_transform(df['language'])
#on mélange le dataframe
shuffled=shuffle(df)
#on sélectionne 80% de notre jeu de données pour l'entrainement
training=shuffled.head(19562)
#on sélectionne 20% de notre jeu de données pour le test
testing=shuffled.tail(4890)
#les catégories de chaque entrée des données d'entrainement
value=training['truthRating']
print(training)
#les catégories réels de chaque entrée des données du test
realvalues=testing['truthRating']
#on élimine les colonnes des catégories (car le modèle va les prédir)
training=training.drop(columns='truthRating')
testing=testing.drop(columns='truthRating')
training=training.drop(columns='id')
testing=testing.drop(columns='id')
training=training.drop(columns='link')
testing=testing.drop(columns='link')
training=training.drop(columns='sourceURL')
testing=testing.drop(columns='sourceURL')
print(value)
#On lance LinearSVC
linSvc=svm.LinearSVC()
#On lui donne les données d'entrainement avec leurs catégories
linSvc.fit(training,value)
#On lui demande de prédir les catégories des données de test
result=linSvc.predict(testing)
print(training)
print(result)
from sklearn.metrics import accuracy_score
#on affiche l'accuracy 
accuracy_score(realvalues,result)

         id   text   date truthRating  author  headline  named_entities_claim  \
8750  16053  16053  16053           1    2784      2911                 12526   
5425  13405  13405  13405           2    1002     20120                  4101   
2881  19935  19935  19935           3    2013        28                  8784   
4054   5475   5475   5475           1     932     14403                 23978   
4045   8510   8510   8510           2    3487     15171                 13944   
...     ...    ...    ...         ...     ...       ...                   ...   
8115  15863  15863  15863           1     932     19302                  5689   
3379   5501   5501   5501           3     932     17803                  5014   
7877  11554  11554  11554           1    3842     18058                 15698   
1462  19493  19493  19493           3     932     18749                 19003   
9777   6002   6002   6002           2     932     12325                 18228   

      named_entities_articl

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.42351738241308795